In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
!pip install -U datasets
!pip install fsspec==2023.9.2
!pip install datasets
!pip install peft
!pip install -U bitsandbytes


In [ ]:
import torch
import io
import pickle
import pandas as pd
from transformers import BlipProcessor, BlipForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, AutoModelForVision2Seq,AutoModel
from peft import LoraConfig  # Import LoRA configuration
from transformers import AutoModelForCausalLM

import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor

model_id = "Qwen/Qwen2-VL-7B-Instruct"

lora_config = LoraConfig(
r=4,
lora_alpha=4,
lora_dropout=0.1,
target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Adjust based on inspected names
bias="none",
init_lora_weights="gaussian"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(model_id)

model.add_adapter(lora_config)
model.enable_adapters()




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [ ]:
from datasets import load_dataset


# Load the vqa-rad dataset
dataset = load_dataset("flaviagiammarino/vqa-rad")
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

# Rename the splits
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']
datasets = {
    'train': train_dataset,
    'valid': valid_dataset
}

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(valid_dataset)}")

README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-eb8844602202be60.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

(…)-00000-of-00001-e5bc3d208bb4deeb.parquet:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/451 [00:00<?, ? examples/s]

Train size: 1434
Validation size: 359


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import random

# Define the VQADataset class
class VQADataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # Assuming it's a PIL image

        return {
            "image": image,
            "query": {"en": question},
            "answers": [answer]
        }


# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.image_token = '<image>'
        self.end_of_utterance_token = '<end_of_utterance>'
        self.pad_token_id = getattr(processor, 'pad_token_id', 0)  # Default to 0 if not found

    def __call__(self, examples):
        texts = []
        for example in examples:
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "text", "text": self.image_token},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer},
                        {"type": "text", "text": self.end_of_utterance_token}
                    ]
                }
            ]

            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)

            if isinstance(text, list):
                text = " ".join(str(item) for item in text)
            else:
                text = str(text)

            texts.append(text.strip())

        text_batch = self.processor(text=texts, return_tensors="pt", padding=True)

        # Prepare the batch dictionary
        batch = {
            "input_ids": text_batch["input_ids"],
            "attention_mask": text_batch["attention_mask"]
        }

        # Create labels
        labels = batch["input_ids"].clone()
        labels[labels == self.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch



dataset1 = load_dataset("flaviagiammarino/vqa-rad")

train_dataset = VQADataset(dataset=datasets['train'], processor=processor)
test_dataset = VQADataset(dataset=dataset1['test'], processor=processor)
valid_dataset = VQADataset(dataset=datasets['valid'], processor=processor)


data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback


training_args = TrainingArguments(
    output_dir="llava",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=100,  # Increased evaluation frequency
    save_steps=100,
    max_steps=300,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    report_to="none",
    optim="paged_adamw_8bit",
    weight_decay=0.05,
    # Add early stopping callback
    # load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset
)

In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,4.338900,0.555147
200,3.645800,0.541681
300,3.260400,0.532858


TrainOutput(global_step=300, training_loss=4.608550752003987, metrics={'train_runtime': 3014.3106, 'train_samples_per_second': 0.796, 'train_steps_per_second': 0.1, 'total_flos': 4801467257413632.0, 'train_loss': 4.608550752003987, 'epoch': 1.6694560669456067})

In [ ]:


# Evaluate the model
results = trainer.evaluate()

print(results)


{'eval_loss': 0.5328576564788818, 'eval_runtime': 207.488, 'eval_samples_per_second': 1.73, 'eval_steps_per_second': 1.73, 'epoch': 1.6694560669456067}


In [ ]:
test_results = trainer.evaluate(test_dataset)

print(test_results)

{'eval_loss': 0.504343569278717, 'eval_runtime': 261.0626, 'eval_samples_per_second': 1.728, 'eval_steps_per_second': 1.728, 'epoch': 1.6694560669456067}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

# Define save path
save_directory = "/content/drive/MyDrive/VQAPublish/VQA/models/qwenqmedR4"


from transformers import AutoConfig

config = AutoConfig.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")  # Use the base model
model.save_pretrained(save_directory)
processor = LlavaProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

print(f"✅ Model saved at: {save_directory}")


✅ Model saved at: /content/drive/MyDrive/VQAPublish/VQA/models/qwenqmedR4
✅ Model saved at: /content/drive/MyDrive/VQAPublish/VQA/models/qwenqmedR4


In [ ]:
from huggingface_hub import HfApi

# Set up the model and tokenizer save path
save_directory = "/content/drive/MyDrive/VQAPublish/VQA/models/qwenqmedR4"
repo_name = "R49JqwenloraMed"
user_name = "Dtarget"

api = HfApi()

api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/R49JqwenloraMed
